In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
#defp('/ml.dev/bin')

from qoreliquid import *
from bitmex import *

import numpy as n
import qgrid as qg # https://github.com/quantopian/qgrid#installation

%pylab inline
from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
import seaborn as sns
sns.set()

p.set_option('display.max_rows', 1000)

pm = PortfolioModeler()
cmc = CoinMarketCap()

In [ ]:
#rng = range(-10,10)
rng = n.linspace(-10,10,100)
x = n.array(rng)
y = x**2 #power(x,2)
"""
dydx = dydx(y,x)
#print x.shape
#print y.shape
#print dydx.shape
#dydx = dydxSym('x**2',x)
#print dydx
plot(x, dydx)
plt.show()
"""

In [ ]:
%load_ext line_profiler
#@profile
def rere():
    import pymongo as mong
    mon = mong.MongoClient()
    mdf = p.DataFrame()
    for i in mon.ql.coins.find():
        df = p.DataFrame(i)
        #df = df.head(10)
        res = list(df.loc[:,'data'.split(' ')]['data'])
        dfi = p.DataFrame(res)
        dfi['q'] = df.index
        dfi = dfi.set_index('q')
        dfi['time'] = df['time']
        dfi['symbolTS'] = map(lambda x: '%s-%s' % (dfi.loc[x, 'symbol'], dfi.loc[x, 'time']), dfi.index)
        dfi = dfi.set_index('symbolTS')
        #print df
        #with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
        #    #print dfi
        #    #print dfi.shape
        mdf = mdf.combine_first(dfi)
        #break
    with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
        #print mdf
        print mdf.shape
%lprun -f rere rere()

In [ ]:
#nmdf = mdf.get_values()
#p.DataFrame(nmdf, columns=mdf.columns, index=mdf.index)
mdf0 = mdf
#mdf0 = mdf0[(mdf['symbol'] == 'BTC') | (mdf['symbol'] == 'ETH')]
#print mdf0.head(5)
val = 'volume'
val = 'marketCap'
mdf0 = mdf0.loc[:,('time symbol %s'%val).split()]#.plot()#.get_values()
mdf0 = mdf0[mdf0.index > 1.512845e+09]
print mdf0.shape
mdf0 = mdf0.drop_duplicates().tail(40000)
mdf0 = mdf0.pivot('time', 'symbol',val)
#mdf0 = normalizeme(mdf0)
#mdf0 = sigmoidme(mdf0)
#mdf0.fillna(0).plot()
#mdf0 = mdf0.sort_values(by'')
#print mdf0
fig, ax = plt.subplots(figsize=(30,40))         # Sample figsize in inches
#sns.heatmap(mdf0.transpose())
#sns.heatmap(mdf0, center=0.5, annot=True, linewidths=0, ax=ax, cmap="YlGnBu")
#mdf0#.corr()
#desc(mdf0)
mdf0

In [ ]:
#print y
mmo = mdf0
for i in mmo.columns:
    y = mmo.loc[:,i].get_values()
    x = n.array(list(mmo.index))
    res = dydx(y, x)
    mmo[i] = res
mmo = mmo.fillna(0)

df = mmo.get_values()
ind = (n.sum(df, 1) > 0)
#print df.shape
#print ind.shape
#print df
#print ind
df2 = df[ind, :]
df2.shape
df2

mmo = mmo.loc[ind, :]

#mmo = mmo.ffill()
mmo = normalizeme(mmo)
mmo = sigmoidme(mmo)
#mmo.plot()
#plt.show()
mmo

In [ ]:
#5000 * ([70 30*[20 80]/100]'/100)

#n.array([70, (n.matrix('20 80', dtype=n.float)/100).A])

In [ ]:
none = desc(df)
pp = sns.pairplot(df)
plt.show()
none

In [ ]:
#  all coins volumePerMarketcap / pcnt scatter plot matrix
from qoreliquid import normalizeme
to = cmc.getAllTokens(tokens=False)
ddd = to.sort_values(by='volumePerMarketcap', ascending=False)
ddd = ddd.loc[:,'marketCap volumePerMarketcap'.split()]
ddd = normalizeme(ddd)
ddd = sigmoidme(ddd)
ddd.plot()
plt.show()
#plt.scatter(ddd['marketCap'], ddd['volumePerMarketcap'])
#plt.show()
ddd = to.loc[:,'pcnt1h pcnt24h pcnt7d'.split(' ')].fillna(0)
ddd = normalizeme(ddd)
ddd = sigmoidme(ddd)
sns.pairplot(ddd, size=10)#, hue='pcnt1h')
plt.show()

none = desc(ddd)
#qg.show_grid(ddd)
qg.show_grid(ddd, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
#cmc.tokensOnOtherExchanges('etherdelta')

In [ ]:
# Poloniex PoC
b = Poloniex()
dfbnb = b.getCurrencies()
print dfbnb['quote']
tokens=True
if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb['base'])
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
#qg.show_grid(dft1s)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()
dft1s

In [ ]:
rm -f /mldev/bin/scraper_cache.sqlite

In [ ]:
# Yobit PoC
yb = Yobit('4E9AB02C1EBD8D3D6DB6E9C416F6889D', '9ab1dfa0ba89b5738396fbd6f0ee7d6d')
dfy = yb.getCurrencies()
dfbnb = dfy
#dfbnb = dfbnb.head(100)
dfbnb.sort_values(by='base', ascending=True)
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000, 'display.max_colwidth', -1):
#    print dfbnb
#print dfbnb['quote']
tokens=True
if tokens: dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
else:      dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
li = list(dfbnb['base'])
#print li
dft = cmc.getAllTokens(tokens=tokens)
#dft1s = dft[dft['marketCap'] <= 1e6]
dft1s = pm.generatePortfolioT1Supply(dft, balance=(5123.83), risk=2)
dft1s = dft1s.loc[li,:]
sortby = 'riskOn'
sortby = 'vb'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
#none = desc(dft1s)
#qg.show_grid(dft1s)
#dft1s.sort_values(by='balanceMarketcapPcnt', ascending=False).loc[:,'pcnt7d'].plot()
dft1s = dft1s[dft1s['volume'] > 0]
qg.show_grid(dft1s.fillna(0), grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [ ]:
# yobit test
mdf = p.DataFrame()
sdf = dft1s.head(20)
for i in list(sdf.index):
    mdf = mdf.combine_first(p.DataFrame(apiRequest('https://yobit.net/api/3','/ticker/%s_btc' % i.lower())))
mdf = mdf.transpose()
mdf['spread'] = mdf['sell'] - mdf['buy']
mdf['spreadA'] = mdf['spread'] / mdf['avg'] * 100
mdf.sort_values(by='vol', ascending=False)
Exchange.getBaseQuote()

In [ ]:
# run: git pull origin [from the etherdelta source code directory to fetch the latest tokens]
#echo '' > etherdelta.tokens.txt; for i in `ls *ejs | cut -d'.' -f 1`; do echo $i >> etherdelta.tokens.txt; echo $i; done

In [ ]:
# Etherdelta PoC
fp = open('/mldev/lib/crypto/ethereum/etherdelta_etherdelta.github.io.github.py.git/tokenGuides/etherdelta.tokens.txt', 'r')
res = fp.read()
fp.close()
li = res.strip().split('\n')
dft = cmc.getAllTokens(tokens=True)
#dft1s = dft[dft['marketCap'] <= 1e6]
#dft = dft[dft['volume'] >= 1e5]
#print dft.sort_values(by='volume', ascending=False)['volume']
#sys.exit()
dft1s = pm.generatePortfolioT1Supply(dft, balance=(5148.36), risk=3.32)
dft1s = visualizePortfolio(dft1s, li)
qg.show_grid(dft1s, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [ ]:
def _color_red_or_green(val):
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

#df.style.applymap(_color_red_or_green)
df.to_html()

In [ ]:
# Binance PoC
rmScraperCache()
b = Binance()
dfbnb = b.getCurrencies()
#dfbnb = dfbnb[dfbnb['quote'] == 'BTC']
dfbnb = dfbnb[dfbnb['quote'] == 'ETH']
li = list(dfbnb['base'])
dft = cmc.getAllTokens(tokens=True)
#dft1s = dft[dft['marketCap'] <= 1e6]
#dft = dft[dft['volume'] >= 1e5]
sortby = 'riskOn'
sortby = 'vb'
#sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
dft1s = dft1s.sort_values(by=sortby, ascending=False)
dft1s = pm.generatePortfolioT1Supply(dft, balance=(5148.36), risk=3.32)
dft1s = visualizePortfolio(dft1s, li)
qg.show_grid(dft1s, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [ ]:
flights = sns.load_dataset("flights")
flights['passengers'] = normalizeme(flights['passengers'])
flights['passengers'] = sigmoidme(flights['passengers'])

fd = flights['month'].drop_duplicates()
di = dict(zip(range(1,13), fd.values))
di = dict(zip(di.values(), di.keys()))
flights['m'] = map(lambda x: di[x], flights['month'])
sns.pairplot(flights.loc[:, 'year passengers'.split(' ')], size=10)#, hue=flights['month'])
plt.show()
#sys.exit()
print flights
flights = flights.pivot("month", "year", "passengers")
flights
#flights.plot()
#plt.show()
ax = sns.heatmap(flights)
plt.show()
print flights

In [ ]:
# show charts from poloniex/binance
df = dft1s.sort_values(by=sortby, ascending=False)
list(df['id'])
yy = True
dfs = genDFP(list(df['id']), max=7, logy=yy, normalize=yy, sigmoid=yy)
dfs

In [ ]:
cm['price_usd'].loc[:,'bitcoin'].sort_values(by='price_usd', ascending=False)#['price_usd']

In [ ]:
cm = clustermap(dfs, verbose=False, figsize=20)

In [ ]:
dft = cmc.getAllTokens(tokens=False)
dft = dft[dft['marketCap'] <= 1e6]#.sort_values(by='balanceMarketcapPcnt', ascending=False)
dft.sort_values(by='volumePerMarketcap', ascending=False)

In [ ]:
dft = cmc.getAllTokens(tokens=False)
#dft = cmc.getAllTokens(tokens=True)
#print dft
#cmc.tokens
#dft = dft[dft['marketCap'] > 1e6]
#dft = dft.loc[port.split(),:] #  port coins [port var set below]
df = pm.generatePortfolioT1Supply(dft, balance=(4129.06), risk=1)
ids = list(df['id'])
#print ids
#dfp = genDFP(ids, max=3)
#dfp
#sortby = 'riskOn'
#sortby = 'vb'
sortby = 'volumePerMarketcap'
#sortby = 'balanceMarketcapPcnt'
#sortby = 'pcnt7d'
#sortby = 'balanceMarketcapPcnt'
df = df.sort_values(by=sortby, ascending=False)#['id']
qg.show_grid(df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})
#df['id']

In [ ]:
list(df['id'])
dfs = genDFP(list(df['id']), max=20)
dfs

In [ ]:
clustermap(dfs, verbose=True, figsize=15)

In [ ]:
df['t1'] = (df['pcnt1h'] + n.min(df['pcnt1h'])+1) *(df['pcnt24h'] + n.min(df['pcnt24h'])+1) * (df['pcnt7d'] + n.min(df['pcnt7d'])+1) / df['marketCap'] / df['price']
dfs = df.sort_values(by='t1', ascending=False)
dfs.loc[:,'pcnt1h pcnt24h pcnt7d'.split()].plot()
dfs = dfs.loc[:,'marketCap pcnt1h pcnt24h pcnt7d t1'.split()]
dfs = normalizeme(dfs)
dfs = sigmoidme(dfs)
#rcParams['figure.figsize'] = 20, 5
#sns.pairplot(dfs, size=10)
#plt.show()
#dfs
dfs = genDFP(list(dfs['id']), max=10)
dfs

In [ ]:
#df = df.head(300)
#df[fi] = map(lambda x: x.decode('utf-8').strip(), df[fi])
#pp = sns.pairplot(df)#, hue=fi)
#plt.show()

In [ ]:
#port = 'PPP XRL BBT SALT PPT TAAS SIFT PAY OMG KIN AIR PLR LINK RHOC CDT DNT RDN POWR CAT ENG PRO KNC SND AVT HMQ MGO ZRX VERI CVC EOS'
#port = 'XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET'

# ib20171128
port = 'RDN AION POWR QSP RHOC DNT LINK ENG FLIXX AVT GMT ZRX CAT HST KNC EOS SALT PRO CVC CREA'

#port = 'CHC QRL BCC XRL RHOC KIN ZRX PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR'
# eth all > $50
port = 'CHC RHOC QRL KIN BNB PPT PRO SALT XRP PPP BRAT TAAS PAY CDT XRL DNT'

# coinmarketcap
#port = 'BTC ETH BTH LTC XRP DASH NEM XMR IOTA NEO'

# bitmex
port = 'BTC BCH DASH ETH ETC LTC XMR XRP XTZ ZEC'

# etoro
#port = 'BTC ETH DASH ETC BCH LTC XRP'

# masternode coins
port = 'CHC DASH NEM PIVX CRW TX EXCL MUE BLOCK BTA ENT SIB BSD DMD KORE BBP'

# git analysis
#port = 'EOS BCH EMC2'

# portPcnt v [model:t1pi commit:51134db]
#port = 'PPP XRL BBT SNC SALT PPT TAAS EXN SIFT PAY OMG KIN AIR RHOC RDN LINK POWR DNT ENG PRO CAT KNC HMQ AVT ZRX CVC EOS'
# delever::currentPortPcnt v [model:t1pi commit:51134db]
#port = 'BNB RHOC PPT PRO PPP SALT BRAT TAAS CDT XRL PAY DNT KIN HMQ GOOD PLR OMG DOM TIX MBRS CVC SKIN POE WINGS LINK BAS BAT CTR CAT STX FUCK BBT TNT BET ICN NIO EOS VERI NEWB E4ROW BQX IXT MSP IND FUN LRC DENT ZRX'
yy = True
dfp = genDFP(port, len(port.split(' ')), logy=yy, normalize=yy, sigmoid=yy)
#dfp

In [ ]:
clustermap(dfp, verbose=False, figsize=15)

In [ ]:
dfm = cmc.glo('ethereum')
dfm

In [ ]:
# Macro Cap [marketcap > 1 billion]
v = DataViz()
sortby=['pcnt7d', 'pcnt24h','pcnt1h']
sortby='riskOn'
sortby='vb'
v.heatmap(maxx=300e9, minn=10e9, usdt=False, figsize=5, sortby=sortby)

In [ ]:
# Macro Cap [marketcap > 1 billion]
v.heatmap(maxx=1e10, minn=1e9, usdt=True, figsize=8, sortby=sortby)

In [ ]:
# Macro Cap [marketcap > 1 billion]
v.heatmap(maxx=10e9, minn=1e9, usdt=True, figsize=5, sortby=sortby)

In [ ]:
# Mid Cap [1 billion > marketcap > 100 million]
v.heatmap(maxx=1e9, minn=100e6, usdt=False, figsize=30, sortby=sortby)

In [ ]:
# Mid Cap [100 million > marketcap > 10 million]
v.heatmap(maxx=100e6, minn=10e6, usdt=True, figsize=60, sortby=sortby)

In [ ]:
# Mid Cap [10 million > marketcap > 1 million]
v.heatmap(maxx=10e6, minn=1e6, usdt=True, figsize=60, sortby=sortby)

In [ ]:
# Mid Cap [1 million > marketcap > 0]
v.heatmap(maxx=1e6, minn=0, usdt=True, figsize=60, sortby=sortby)

In [ ]:
# Mid Cap [1 million > marketcap > 0]
v.heatmap(maxx=1e5, minn=0, usdt=True, figsize=10, sortby=sortby)

In [ ]:
li = list(df['id'].head(3))
viewCharts(li)

In [ ]:
# show portfolio  charts
viewCharts('PPP XRL BBT SALT PPT TAAS SIFT PAY OMG KIN AIR PLR LINK RHOC CDT DNT RDN POWR CAT ENG PRO KNC SND AVT HMQ MGO ZRX VERI CVC EOS')

In [ ]:
# show portfolio eth1 charts
viewCharts('XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET')

In [ ]:
# show portfolio eth2 charts
viewCharts('CHC QRL BCC XRL RHOC KIN PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR')

In [ ]:
nu = 10
for i in list(df.head(nu)['id']):
    cmc.glo(i)
    plt.show()
print '=-=-=-=-=-=-=-=-'
print '=-=-=-=-=-=-=-=-'
for i in list(df.tail(nu)['id']):
    cmc.glo(i)
    plt.show()

In [ ]:
#for i in 'bitcoin ethereum kin'.split(' '):
# b3 == pump n dump example
for i in 'b3coin'.split(' '):
    cmc.glo(i)

In [ ]:
import pandas as p
url = 'https://etherscan.io/txsInternal?a=0xea3f1b409f5e9211eed087c9f5f621d1249b3e7b'
from qore import *
xp = XPath()
xcols = {
    #'to'      : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[6]/span/a/text()',
    'value'   : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[7]/text()',
}
res = xp.xpath2df(url, xcols)
qg.show_grid(p.DataFrame(res))

In [ ]:
#port = 'CHC LBTC BTCRED ARN'
port = 'CHC QRL PPT SALT BNB KIN CDT PRO PPP TAAS'
port = 'PAY GNT PPT QTM LTC'
# portPcnt v [model:t1pi commit:51134db]
#print dft1s.sort_values(by=sortby, ascending=False)['id'].head(10)
#port = ' '.join(list(dft1s.sort_values(by=sortby, ascending=False)['id'].head(10)))
yy = True
dfp = genDFP(port, len(port.split(' ')), logy=yy, normalize=yy, sigmoid=yy)
#dfp

In [ ]:
# dendrogram
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sc

dists = np.array([[0,2,1,4],[2,0,3,5],[1,3,0,6],[4,5,6,0]])
l = ['a','b','c','b']
df = p.DataFrame(dists)
print df#.plot()
Z = sc.linkage(dists, method='complete')
d = sc.dendrogram(Z, labels=l)
label_colors = {'a': 'r', 'b': 'g', 'c': 'm'}
ax = plt.gca()
xlbls = ax.get_xmajorticklabels()
for i in range(len(xlbls)):
    xlbls[i].set_color(label_colors[xlbls[i].get_text()])
plt.show()

In [ ]:
# Load the brain networks example dataset
df = sns.load_dataset("brain_networks", header=[0, 1, 2], index_col=0)

# Select a subset of the networks
used_networks = [1, 5, 6, 7, 8, 12, 13, 17]
used_columns = (df.columns.get_level_values("network")
                          .astype(int)
                          .isin(used_networks))
df = df.loc[:, used_columns]

# Create a categorical palette to identify the networks
network_pal = sns.husl_palette(8, s=.45)
#print network_pal
network_lut = dict(zip(map(str, used_networks), network_pal))

# Convert the palette to vectors that will be drawn on the side of the matrix
networks = df.columns.get_level_values("network")
network_colors = p.Series(networks, index=df.columns).map(network_lut)

# Draw the full plot
sns.clustermap(df.corr(), center=0, cmap="vlag",
               row_colors=network_colors, col_colors=network_colors,
               linewidths=.75, figsize=(13, 13))
#dfd = desc(df)
#df
df#.corr()

In [ ]:
# 0024 mergerTraderSplitter
mergerTraderSplitter([13.7, 55.3], 0.3347784)